### "Simplification" attemps with OSMnx

For test (continents) cities [FUA ID]:
* (Africa) Douala [809]
* (Oceania) Auckland [869]
* (Asia) Aleppo [1133]
* (Europe) Liège [1656]
* (South America) Bucaramanga [4617]
* (North America) Salt Lake City [4881]

In [24]:
# import libraries
import json
import os

import contextily as cx
import cv2
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import osmnx as ox
from shapely.geometry import Point

**Read in meta data**

In [4]:
# read in sample metadata
sample = gpd.read_parquet("../data/sample.parquet")
sample.head(2)

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry,continent,iso_a3
305,9129.0,1.0,8078,Gonda,1.0,IND,India,66.0,29.0,1.074100e+06,1.066419e+06,7680.678101,"POLYGON ((81.98398 27.19657, 81.99471 27.19657...",Asia,IND
91,7578.0,6.0,10577;10581;10583;10596;10605;10607,Chongqing,1.0,CHN,China,2267.0,618.0,6.036834e+06,5.157726e+06,879107.861057,"POLYGON ((106.23972 29.52328, 106.19622 29.523...",Asia,CHN


In [1]:
# dict of cityname : fua ID
citydict = {
    "Douala": 809,
    "Auckland": 869,
    "Liège": 1656,
    "Aleppo": 1133,
    "Bucaramanga": 4617,
    "Salt Lake City": 4881,
}

**Read in data for example city**

In [5]:
# read in data for example city: Liege
cityname = "Liège"
fua = citydict[cityname]
gdf = gpd.read_parquet(f"../data/{fua}/roads_osm.parquet")
gdf = gdf[["highway", "geometry"]]
gdf = gdf.reset_index(drop=True)
G = momepy.gdf_to_nx(gdf_network=gdf, approach="primal", directed=True, osmnx_like=True)

**Simplify graph (in OSMNnx terms, i.e. remove interstitial nodes)**

In [6]:
G_simp = ox.simplify_graph(G)

In [8]:
# check crs, needs to be a projected one
G_simp.graph["crs"]

<Projected CRS: {"$schema": "https://proj.org/schemas/v0.5/projjso ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 31N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

**Consolidate nodes (test different thresholds)**

In [14]:
cons_dict = {}
for tol in range(1, 21):
    print(f"Consolidating with tolerance {tol}m")
    # consolidate graph
    G_cons = ox.consolidate_intersections(
        G=G_simp,
        tolerance=tol,
        rebuild_graph=True,
        dead_ends=True,
        reconnect_edges=True,
    )
    # derive consolidated edges
    edges_cons = ox.graph_to_gdfs(G=G_cons, nodes=False, edges=True)
    # save in cons_dict
    cons_dict[tol] = {}
    cons_dict[tol]["graph"] = G_cons
    cons_dict[tol]["edges"] = edges_cons

Consolidating with tolerance 1m
Consolidating with tolerance 2m
Consolidating with tolerance 3m
Consolidating with tolerance 4m
Consolidating with tolerance 5m
Consolidating with tolerance 6m
Consolidating with tolerance 7m
Consolidating with tolerance 8m
Consolidating with tolerance 9m
Consolidating with tolerance 10m
Consolidating with tolerance 11m
Consolidating with tolerance 12m
Consolidating with tolerance 13m
Consolidating with tolerance 14m
Consolidating with tolerance 15m
Consolidating with tolerance 16m
Consolidating with tolerance 17m
Consolidating with tolerance 18m
Consolidating with tolerance 19m
Consolidating with tolerance 20m


In [36]:
for tol in cons_dict:
    cons_dict[tol]["nodes"] = ox.graph_to_gdfs(
        G=cons_dict[tol]["graph"], nodes=True, edges=False
    )

**Find examples** (see nb `usecases.ipynb`)

**Check what happens for examples / use cases for different consolidation thresholds**

Make plots and videos for each use case, for gradually increasing simplification threshold

In [53]:
with open(f"../usecases/{fua}/points.json") as infile:
    points = json.load(infile)

In [ ]:
package = "osmnx"

In [44]:
# MAKE PLOTS

for v in points.values():
    mypoint = v["coords"]
    myclass = v["class"]

    # make subfolder for plot saving
    os.makedirs(f"../usecases/{fua}/{package}/{myclass}/", exist_ok=True)

    # get center frame (for clipping)
    center = gpd.GeoDataFrame(geometry=[Point(mypoint)], crs="epsg:4326")
    center = center.to_crs(gdf.crs)
    center = center.buffer(250, cap_style=3)

    # for each tolerance threshold,
    for tol in cons_dict:
        # make a plot
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))

        edges = cons_dict[tol]["edges"]
        nodes = cons_dict[tol]["nodes"]

        # clip geometries to box
        edges_clipped = edges.copy()
        nodes_clipped = nodes.copy()
        edges_clipped = edges_clipped.clip(center)
        nodes_clipped = nodes_clipped.clip(center)

        # plot
        edges_clipped.plot(ax=ax, zorder=1, color="black", linewidth=2)
        nodes_clipped.plot(ax=ax, zorder=2, color="red", markersize=15, alpha=0.9)
        cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Voyager, crs=gdf.crs)
        ax.set_axis_off()
        ax.set_title(f"Tolerance {tol}m")
        plt.tight_layout()

        # save to subfolder
        fill_tol = f"{tol:03d}"
        fig.savefig(f"../usecases/{fua}/{package}/{myclass}/{fill_tol}.png", dpi=300)
        plt.close()

In [54]:
# MAKE VIDEOS

for v in points.values():
    myclass = v["class"]

    fps = 1
    img_folder_name = f"../usecases/{fua}/{package}/{myclass}/"
    images = sorted(
        [img for img in os.listdir(img_folder_name) if img.endswith(".png")]
    )
    video_name = f"../usecases/{fua}/{myclass}.mp4"
    frame = cv2.imread(os.path.join(img_folder_name, images[0]))
    height, width, layers = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video = cv2.VideoWriter(video_name, fourcc, fps, (width, height))
    for image in images:
        fp = os.path.join(img_folder_name, image)
        video.write(
            cv2.resize(cv2.imread(fp), (width, height)),
        )
    cv2.destroyAllWindows()
    video.release()